In [23]:
import re
import ast
import pandas as pd
import numpy as np
import time
from datetime import datetime

gameplay = pd.DataFrame()

clean = open('bronze_raw.txt').read().replace('\n', '')
champs = clean.split("|")

#Parse data 
for champ in champs[0:-1]:
    
    df = pd.DataFrame()
    df1 = pd.DataFrame()
    df2 = pd.DataFrame()
    df3 = pd.DataFrame()
    
    temp_champ = champ
    graphs = re.findall ( 'data:(.*?)lines:', temp_champ, re.MULTILINE)

    name = temp_champ.split('^')[0]
    #print(name)
    dt = datetime.strptime(temp_champ.split('^')[4], 
                                        '                %b %Y            ')
    release = time.mktime(dt.timetuple())

    s = graphs[0]
    data_pop = ast.literal_eval(s[1:-25])

    data = np.array(data_pop)
    df1['date'] = data[:,0]/1000
    df1['popularity'] = data[:,1]

    s = graphs[1]
    data_wr = ast.literal_eval(s[1:-25])
    data2 = np.array(data_wr)
    df2['date'] = data2[:,0]/1000
    df2['winrate'] = data2[:,1]

    s = graphs[2]
    data_br = ast.literal_eval(s[1:-25])
    data3 = np.array(data_br)
    df3['date'] = data3[:,0]/1000
    df3['banrate'] = data3[:,1]

    df = pd.merge(pd.merge(df1, df2, on='date'), df3, on='date', how='outer')
    df['name'] = name
    df['release'] = release
    gameplay = pd.concat([gameplay, df], axis=0)
    
gameplay.sort_values('date', inplace=True)
gameplay.reset_index(drop=True, inplace = True)
gameplay.head()

,date,popularity,winrate,banrate,name,release
0,1.394525e+09,4.17,50.20,0.34,aatrox,1.370045e+09
1,1.394525e+09,4.46,48.71,0.15,ryze,1.233446e+09
2,1.394525e+09,5.26,50.66,0.10,sejuani,1.325376e+09
3,1.394525e+09,4.52,48.77,0.31,diana,1.343779e+09
4,1.394525e+09,4.69,48.37,0.19,shaco,1.254355e+09


In [18]:
dates = gameplay.date.unique()
dates

array([  1.39452533e+09,   1.39514040e+09,   1.39576680e+09,
         1.39641120e+09,   1.39717081e+09,   1.39780800e+09,
         1.39848840e+09,   1.39932233e+09,   1.39992841e+09,
         1.40055990e+09,   1.40119170e+09,   1.40183281e+09,
         1.40247151e+09,   1.40312400e+09,   1.40380500e+09,
         1.40441610e+09,   1.40503200e+09,   1.40565030e+09,
         1.40631180e+09,   1.40722530e+09,   1.40786310e+09,
         1.40854590e+09,   1.40918640e+09,   1.40982514e+09,
         1.41047196e+09,   1.41109710e+09,   1.41177000e+09,
         1.41242520e+09,   1.41321824e+09,   1.41383694e+09,
         1.41444360e+09,   1.41505560e+09,   1.41573600e+09,
         1.41634440e+09,   1.41696000e+09,   1.41780600e+09,
         1.41880681e+09,   1.41955920e+09,   1.42024243e+09,
         1.42097814e+09,   1.42165441e+09,   1.42234560e+09,
         1.42238228e+09,   1.42321320e+09,   1.42385400e+09,
         1.42453440e+09,   1.42521480e+09,   1.42584121e+09,
         1.42646400e+09,

In [22]:
dfp = pd.read_csv('patchnotes.txt', sep='|', names=['patch', 'date', 'buffs', 'nerfs'])
dfp.fillna(' ', inplace=True)
dfp = dfp[dfp['buffs'].str.contains('buffs')]
dfp['buffs'] = dfp['buffs'].str.replace('buffs:', '')
dfp['buffs'] = dfp['buffs'].str.replace('[:;.]+', '')
dfp['nerfs'] = dfp['nerfs'].str.replace('nerfs:', '')
dfp['nerfs'] = dfp['nerfs'].str.replace('[:.;]+', '')
dfp.sort_values('date', inplace=True)
dfp.reset_index(drop=True, inplace = True)
dfp.head()

,patch,date,buffs,nerfs
0,1.0,1.355431e+09,"chogath,diana,rengar,teemo,zyra,brutalizer,bla...","elise,fiora,masteryi"
1,2.0,1.358293e+09,releasedyet,"amumu,blitzcrank,evelynn,ezreal,katarina,khazi..."
2,3.0,1.359660e+09,"blitzcrank,chogath,ezreal,katarina,missfortune...","heimerdinger,kennen,nasus,sion,tristana,trynda..."
3,3.0,1.360786e+09,,"jarvaniv,lulu"
4,3.5,1.365538e+09,"corki,karma,morgana,nami,nautilus,quinn","akali,kayle,lux,volibear"


In [35]:
for index, row in dfp.iterrows():
    if index != 0:
        print(dfp['date'].iloc[index-1])
        print(row['date'])
        gameplay[(gameplay['date'] < row['date']) & 
                 (gameplay['date'] > dfp['date'].iloc[index-1]) &
                (gameplay['name']=='rumble')]['change'] = 'test'

1355431380.0
1358293320.0
1358293320.0
1359660300.0
1359660300.0
1360786320.0
1360786320.0
1365538080.0
1365538080.0
1367271120.0
1367271120.0
1368648840.0
1368648840.0
1370897040.0
1370897040.0
1373404680.0
1373404680.0
1375224720.0
1375224720.0
1376434680.0
1376434680.0
1378333200.0
1378333200.0
1380578700.0
1380578700.0
1383090840.0
1383090840.0
1386889440.0
1386889440.0
1389736920.0
1389736920.0
1392059400.0
1392059400.0
1393454040.0
1393454040.0
1395174540.0
1395174540.0
1396774980.0
1396774980.0
1398118020.0
1398118020.0
1399493700.0
1399493700.0
1400710620.0


/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1400710620.0
1401836040.0
1401836040.0
1403042700.0
1403042700.0
1404246180.0
1404246180.0
1405454940.0
1405454940.0
1406670240.0
1406670240.0
1406763480.0
1406763480.0
1407888840.0
1407888840.0
1410286680.0
1410286680.0
1411597260.0
1411597260.0
1412816220.0
1412816220.0
1415143800.0
1415143800.0
1416438180.0
1416438180.0
1418156880.0
1418156880.0
1421254740.0
1421254740.0
1422386100.0
1422386100.0
1423597740.0
1423597740.0
1424805420.0
1424805420.0
1426097940.0
1426097940.0
1427236500.0
1427236500.0
1428441000.0
1428441000.0
1430269260.0
1430269260.0
1431535620.0
1431535620.0
1432747920.0
1432747920.0
1433867580.0
1433867580.0
1435102380.0
1435102380.0
1436301000.0
1436301000.0
1437499320.0
1437499320.0
1438727340.0
1438727340.0
1440020520.0
1440020520.0
1441183920.0
1441183920.0
1442372280.0
1442372280.0
1443566580.0
1443566580.0
1444794300.0
1444794300.0
1446071940.0
1446071940.0
1447231980.0
1447231980.0
1448297760.0
1448297760.0
1449608400.0
1449608400.0
1453105560.0
1453105560.0

In [34]:
gameplay[(gameplay['date'] < row['date']) & 
                 (gameplay['date'] > dfp['date'].iloc[index-1]) &
                (gameplay['name']=='rumble')]

,date,popularity,winrate,banrate,name,release


In [36]:
gameplay.head(50)

,date,popularity,winrate,banrate,name,release
0,1.394525e+09,4.17,50.20,0.34,aatrox,1.370045e+09
1,1.394525e+09,4.46,48.71,0.15,ryze,1.233446e+09
2,1.394525e+09,5.26,50.66,0.10,sejuani,1.325376e+09
3,1.394525e+09,4.52,48.77,0.31,diana,1.343779e+09
4,1.394525e+09,4.69,48.37,0.19,shaco,1.254355e+09
5,1.394525e+09,2.09,49.18,0.20,zilean,1.238544e+09
6,1.394525e+09,5.13,46.64,0.58,shen,1.267402e+09
7,1.394525e+09,11.19,49.32,8.45,shyvana,1.320106e+09
8,1.394525e+09,7.32,49.55,0.56,darius,1.335830e+09
9,1.394525e+09,2.70,49.43,0.12,singed,1.238544e+09


In [7]:
# dt = datetime.fromtimestamp(gameplay['release'].iloc[0])
dt = datetime.fromtimestamp(gameplay['date'].iloc[0])
print(dt.ctime())
dt = datetime.fromtimestamp(gameplay['date'].iloc[1])
print(dt.ctime())

Tue Mar 11 08:08:51 2014
Tue Mar 18 11:00:01 2014


In [ ]:
gameplay.iloc[1000]

In [ ]:
import seaborn as sb
%matplotlib inline

sb.distplot(df['winrate'])

In [ ]:
sb.tsplot(df['winrate'], value="Win Rate",)

ImportError: No module named 'nltk'